#### 一. 何为词典
1. 词典是一个集合, 其中个元素由"关键码"和"数据"组成
2. 逻辑上讲, 词典元素中的关键码可以是相同的, 且无大小关系. 但数据结构中的'词典', 其关键码往往有次序关系  
 例如: `java.util.TreeMap`是基于红黑树实现的词典结构, 这类实现方法, 都预示着"关键码可以比大小". 故其并非严格意义的词典


### 1.1 跳转表

#### 一. 跳转表的定义  
1. 调准表是一种高效的词典结构, 其查询和维护的平均时间复杂度仅需要$O(logn)$
2. 跳转表的宏观结构为 : 内部由横向分层, 纵向层对一个元素之间存在引用的多个顺序表结构. 如下图所示  
    1. 其中最底层为$S_0$层, 顶层为$S_n$层  
    2. 同层节点之间按照`次序`形成前驱后继关系. 每层顺序表, 为简化操作, 都定义有头哨兵, 尾哨兵   
    3. 不同层次的节点按纵向组成塔(tower), 同一塔内的节点以高度为序, 也可以定义前驱后继关系  
    4. 高层的顺序表总是底层顺序表的子集. 特别的$S_0$层包括全部词条, 而$S_0$层除了包含头尾节点外不包含任何实际词条.    
    
<img src='img/skiplist.png' heihgt='65%' width='65%'>
   
#### 二. 四联表定义  
 1. 按照上述定义, 跳转表的每层的内部节点, 在水平和垂直方向上都有定义. 是4个方向的引用. 所以将支持这种连接方式的顺序表称为四联表  
  即: 跳转表的每层都是一个四联表  
 2. 四联表是一个节点为四联接点的顺序表, 因此四联表要继承顺序表, 来使用其`empty`,`insert`,`find`等方法


In [27]:
class QuadlistNode(object):
    '''四联表节点'''
    def __init__(self,entry=None,pred=None,succ=None,above=None,below=None):
        self.entry = entry
        self.pred = pred
        self.succ = succ
        self.above = above
        self.below = below
        
    def insertAsSuccAbove(self,e,bnode=None):
        '''将词条e作为当前节点的后继, bnode的上临建立关系'''
        new_node = QuadlistNode(e,self,self.succ,None,bnode)
        self.succ.pred = new_node
        self.succ = new_node
        if bnode is not None:
            bnode.above = new_node
        return new_node
        
class Entry(object):
    def __init__(self,k,v):
        self.key = k
        self.value = v

In [28]:
class QuadList(object):
    def __init__(self):
        '''四联表'''
        self.header = QuadlistNode()  #头哨兵,尾哨兵
        self.trailer = QuadlistNode()   
        #横向连接哨兵
        self.header.succ = self.trailer  
        self.header.pred = None
        self.trailer.pred = self.header
        self.trailer.succ = None
        #纵向连接哨兵
        self.header.above = None 
        self.header.below = None 
        self.trailer.above = None 
        self.trailer.below = None 
        self.size = 0
        
    def first(self):
        return self.header.succ
    
    def last(self):
        return self.trailer.pred
    
    def valid(self,node): 
        '''判断node位置是否合法'''
        return (node is not None) and (self.trailer is not node) and (self.header is not node)
    
    def empty(self):
        return self.size<=0
    
    def insertSuccAbove(self,e,pnode,bnode=None):
        '''在pnode的后面,bnode的上面插入节点'''
        self.size = self.size + 1
        return pnode.insertAsSuccAbove(e,bnode)
        
    def remove(self,pnode):
        '''从四联表删除节点pnode'''
        pnode.pred.succ = pnode.succ
        pnode.succ.pred = pnode.pred
        self.size = self.size-1
        return pnode.entry
    
#     def traverse(self,visit):
#         '''遍历四联表'''
#         node = self.header.succ
#         while(node is not self.tailer):
#             visit(node.data)
#             node = node.succ

#### 三. 跳转表定义  
1. 因为跳转表也是一个顺序表, 只是每个节点变成了四联表, 使得顺序表变为二维结构  
2. 跳转表的size  
  1. 跳转表的大小为最底层$S_0$层元素的size  
  2. 跳转表在插入四联表时, 使用前插 (`insertBefore`),让越高层的四联表出现在跳转表的越靠前的位置

#### 四. 跳转表查找  
1. 从最高层四联开始, 逐个节点查找.  
2. 如果查找不成功, 到相应位置(塔)的下层四联表继续查找, 直到查找了所有层的四联表
  
#### 五. 跳转表插入新词条
1. 查找新词条应该被插入的位置. 这个位置总是返回最低层四联表的一个节点, 在该节点后面插入新词条作为"塔"的基座  
2. 掷硬币, 假设若为正面, 则在上层四联表也创建该节点, 并组织好"四联关系"

#### 六. 跳转表删除
首先查找关键码, 找到后从塔顶删除

In [71]:
from sequencelist import List,ListNode

class SkipList(List):
    def __init__(self):
        self.header = ListNode()  # 头哨兵
        self.tailer = ListNode()    # 尾哨兵
        self.header.succ = self.tailer  # 连接首尾
        self.tailer.pred = self.header
        self.header.pred = None
        self.tailer.succ = None
        self.size = 0
        
    def size(self):
        '''内部四联表个数为0,则跳转表个数为0; 否则取S0层的size为元素个数'''
        return 0 if self.empty() else self.last().data.size
    
    def level(self):
        '''跳转表层数为内部四联表的个数'''
        return self.size
    
    def __skip_search(self,qlist,pnode,k):
        '''跳转表查找
           入口: qlist为顶层四联表, pnode为qlist的首节点(qlist.first()),k为待查找的关键码
           出口: 若查找成功, pnode为关键码所属塔的顶部节点, qlist为pnode所属四联表
                若查找失败, pnode为所属塔的基座. 该塔为不大于k的最大且最右关键码, qlist为空
        '''
        while 1:
            # 在本层四联表内查找关键码, 直到找出更大的key或溢出至tailer 
            # (pnode.succ is not None)让pnode变成trailer后面的None节点
            while (pnode.succ is not None) and (pnode.entry.key<=k):  
                pnode = pnode.succ  
            # 倒退一步, 看是否命中
            pnode = pnode.pred
            # 条件(pnode.pred is not None)避免pnode为四联表中的header节点
            if(pnode.pred is not None) and (pnode.entry.key==k):
                return (True,pnode.entry.value,pnode)   
            else:
                qlist = qlist.succ # 转至下一层
                if qlist.succ is None: #qlist已经是tailer, 查找失败, 返回pnode处于最底层,带插入位置为pnode的后继
                    return (False,None,pnode)
                if pnode.pred is not None : # pnode不是header则可直接取其below, 因为header.below=None
                    pnode = pnode.below
                else:
                    pnode = qlist.data.first() 
                    
    def get(self,key):
        qlist = self.first()   # 顶层
        pnode = qlist.data.first()  # 首节点
        res = self.__skip_search(qlist,pnode,key)
        return None if (res[0] is False) else res[1]
    
    def put(self,k,v):
        '''为简化操作, 若已存在关键码, 则新插入失败'''
        entry = Entry(k,v)
        #若跳转表为空, 则需要先创建第一个四联表
        if self.empty():  
            self.insertAsFirst(QuadList())
        #从顶层四联表的首节点出发
        qlist = self.first()
        pnode = qlist.data.first()
        (flag,value,pnode) = self.__skip_search(qlist,pnode,k)
        if flag:  #查找成功,直接返回
            return False
        #从s0层开始插入
        qlist = self.last()
        bnode = qlist.data.insertSuccAbove(entry,pnode)  #查找失败时,返回最底层基座节点, 所以插入时应该在返回节点的后面插入
        # 投硬币,决定是否增长一层(伪随机数为奇数,增长一层)
        import random
#         while random.randint(1,10)%2 == 1:
        for i in range(4):
            #要想在上一层插入节点, 需要找到上层能插入节点的前驱
            while qlist.data.valid(pnode) and (pnode.above is None):
                pnode = pnode.pred
            if not qlist.data.valid(pnode): #若找到的前驱已经是header节点, 则上层带插入位置pnode
                if qlist is self.first():
                    self.insertAsFirst(QuadList()) #如果已经是顶层,则新增一层
                pnode = qlist.pred.data.header
            else:
                pnode = pnode.above    
            qlist = qlist.pred
            bnode = qlist.data.insertSuccAbove(entry,pnode,bnode)
        return True
    
    def remove(self,key):
        '''删除词条'''
        if self.empty():
            return False
        qlist = self.first() #从顶层首节点开始
        pnode = qlist.data.first()
        (flag,data,pnode) = self.__skip_search(qlist,pnode,key)
        if not flag:
            return False
        while qlist.succ is not None:
            lower = pnode.below #记录下层节点
            qlist.data.remove(pnode)
            pnode = lower
            qlist = qlist.succ
            
        #删除不含词条的顶层四联表
        while (not self.empty()) and (self.first().data.empty()):
            self.remove(self.first())
        return True
    
#     def getall(self):
#         lowest_layer = self.last().data
#         print lowest_layer
#         def visit(data):
#             print data
#         lowest_layer.traverse(visit)

In [72]:
if __name__=="__main__":
    l = SkipList()
    l.put(1,'a')
    l.put(2,'b')
    print l.get(1)
    print l.get(2)
    l.remove(1)
    print l.get(1)
    print l.get(2)

a
b
None
b
